In [1]:
%%capture
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip uninstall cupy-cuda115 -y
!pip install cupy-cuda112
!pip install anndata

In [2]:
import cupy
import logging
import os
import gc
import time

import anndata as ad
import numpy as np

from datetime import datetime
from google.colab import drive
from cuml.decomposition import TruncatedSVD
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from cuml.kernel_ridge import KernelRidge
from glob import glob
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)

In [3]:
## MOUNT DRIVE
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
## DATA DIRECTORY CONSTANT
DATA_DIR = '/content/gdrive/My Drive/Thesis/dance/dance/data/'
PREDS_DIR = '/content/gdrive/My Drive/Thesis/dance/dance/data/predictions'
SUBTASK = 'mouse_liver_cite_fltr_denoised_transp_rna'
RANDOM_SEED = 123

In [5]:
## FILE LOAD AND TRAIN VAL SPLIT CREATION
# Load anndata files
input_train_mod1 = ad.read_h5ad(os.path.join(
    DATA_DIR, SUBTASK, f'{SUBTASK}.censor_dataset.output_train_mod1.h5ad'))
input_train_mod2 = ad.read_h5ad(os.path.join(
    DATA_DIR, SUBTASK, f'{SUBTASK}.censor_dataset.output_train_mod2.h5ad'))
input_test_mod1 = ad.read_h5ad(os.path.join(
    DATA_DIR, SUBTASK, f'{SUBTASK}.censor_dataset.output_test_mod1.h5ad'))
input_test_mod2 = ad.read_h5ad(os.path.join(
    DATA_DIR, SUBTASK, f'{SUBTASK}.censor_dataset.output_test_mod2.h5ad'))

In [6]:
# Create random indexes for splits if task is not altstrat
if 'altstrat' not in SUBTASK:
  if RANDOM_SEED:
    np.random.seed(RANDOM_SEED)
  idx = np.random.permutation(input_train_mod1.shape[0])
  train_idx = idx[:int(idx.shape[0] * 0.88889)]
  val_idx = idx[int(idx.shape[0] * 0.88889):]

else:
  train_bool = input_train_mod1.obs['cell_type'] != 'MkP'
  val_bool = input_train_mod1.obs['cell_type'] == 'MkP'

  train_idx = input_train_mod1.obs.index[train_bool]
  val_idx = input_train_mod1.obs.index[val_bool]

train_mod1 = input_train_mod1[train_idx,:]
train_mod2 = input_train_mod2[train_idx,:]

val_mod1 = input_train_mod1[val_idx,:]
val_mod2 = input_train_mod2[val_idx,:]

test_mod1 = input_test_mod1
test_mod2 = input_test_mod2

In [7]:
pred_dimx = test_mod1.shape[0]
pred_dimy = train_mod2.shape[1]

feature_obs = train_mod1.obs
gs_obs = train_mod2.obs

batches = train_mod1.obs.batch.unique().tolist()
batch_len = len(batches)

In [8]:
concat_train = ad.concat(
    {"train": train_mod1, "test": test_mod1},
    axis=0,
    join="outer",
    label="group",
    fill_value=0,
    index_unique="-"
)

In [9]:
logging.info('Determine parameters by the modalities')
n_mod1, n_mod2, scale, alpha = (300, 70, 10, 0.2)
logging.info(f"{n_mod1}, {n_mod2}, {scale}, {alpha}")

In [10]:
X = cupy.asarray(concat_train.X.toarray(), order='F')
y = cupy.asarray(train_mod2.X.toarray(), order='F')

In [11]:
start = time.process_time()

## DIMENSIONALITY REDUCTION
def get_mask(ds, val):
    r = ds == val
    return r.values

embedder_mod1 = TruncatedSVD(n_components=n_mod1, random_state=RANDOM_SEED)
mod1_pca = embedder_mod1.fit_transform(X).astype(np.float32)
train_matrix = mod1_pca[get_mask(concat_train.obs['group'],'train')]
test_matrix = mod1_pca[get_mask(concat_train.obs['group'],'test')]

embedder_mod2 = TruncatedSVD(n_components=n_mod2, random_state=RANDOM_SEED)
train_gs = embedder_mod2.fit_transform(y).astype(np.float32)


del concat_train
del input_train_mod1
del input_train_mod2
del input_test_mod1
del X, y
gc.collect()


logging.info('Running normalization ...')
train_sd = np.std(train_matrix, axis=1).reshape(-1, 1)
train_sd[train_sd == 0] = 1
train_norm = (train_matrix - np.mean(train_matrix, axis=1).reshape(-1, 1)) / train_sd
train_norm = train_norm.astype(np.float32)
del train_matrix


test_sd = np.std(test_matrix, axis=1).reshape(-1, 1)
test_sd[test_sd == 0] = 1
test_norm = (test_matrix - np.mean(test_matrix, axis=1).reshape(-1, 1)) / test_sd
test_norm = test_norm.astype(np.float32)
del test_matrix
gc.collect()


print('Running KRR model ...')
y_pred = cupy.zeros((pred_dimx, pred_dimy), dtype=np.float32)
cupy.random.seed(123)
emb = embedder_mod2.components_.to_output('cupy')
gc.collect()

seeds = range(10)
for seed in tqdm(seeds):
    gex_tr,_,adt_tr,_ = train_test_split(train_norm, 
                                         train_gs,
                                         train_size=0.5, 
                                         random_state=seed)
    gc.collect()
    krr = KernelRidge(alpha=alpha, kernel='rbf')
    print('Fitting KRR ... ')
    krr.fit(gex_tr, adt_tr)
    gc.collect()
    y_pred += krr.predict(test_norm) @ emb
    gc.collect()

np.clip(y_pred, a_min=0, a_max=None, out=y_pred)
y_pred /= len(seeds)

end = time.process_time()
print('Time elapsed:%.2f seconds'%(end-start))

Running KRR model ...


  0%|          | 0/10 [00:00<?, ?it/s]

Fitting KRR ... 


 10%|█         | 1/10 [00:09<01:28,  9.86s/it]

Fitting KRR ... 


...

Fitting KRR ... 


 90%|█████████ | 9/10 [00:12<00:00,  1.86it/s]

Fitting KRR ... 


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


Time elapsed:18.53 seconds


In [12]:
np.save(os.path.join(PREDS_DIR, f'dengkw_{SUBTASK}'), y_pred.get())